# Q8
Tree Based Models - Q08- 09/July

Consider the dataset from Q07 (credit_data). 
https://drive.google.com/drive/folders/1Jl8iDu7nGmrqCECbrLqmVafgwE5PYfiU

The decision trees overfit the training data. To avoid overfitting we can do a few things.

Train decision tree using sklearn DecisionTreeClassifier() with the following condition and check how train and test accuracy vary:

    - Case 1) max_depth  = 5, 6, and 7
    - Case 2) min_samples_split = 10, 15 & 20
    - case 3) max_features = “sqrt”, “log2”

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("01_credit_data.csv")
df.head(2)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,T
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,f,g,202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,f,g,43,560,+


In [3]:
cat_var_list = ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']
num_vrb_list = ['A2', 'A3', 'A8', 'A11', 'A14', 'A15']

In [4]:
#df[num_vrb_list] = df[num_vrb_list].astype(float)
for col in  num_vrb_list:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(axis=0)

In [5]:
df_encoded = pd.get_dummies(df, columns = cat_var_list, prefix_sep='_', drop_first=True)
df_encoded['target'] = np.where(df_encoded['T'] == '+', 1, 0)
#df_encoded.head()

In [6]:
# varify encoding
pd.crosstab(df_encoded['T'], df_encoded['target'])

target,0,1
T,,
+,0,299
-,367,0


In [7]:
df_encoded.drop('T', axis=1, inplace=True)

In [8]:
df_train, df_test = train_test_split(df_encoded, test_size=0.25, random_state=25)
print(df_train.shape)
print(df_test.shape)

(499, 41)
(167, 41)


In [9]:
# create a dictionary, to create cases, as mentioned in quiz
#- Case 1) max_depth  = 5, 6, and 7
#- Case 2) min_samples_split = 10, 15 & 20
#- case 3) max_features = “sqrt”, “log2”
dict_comb = {'case_1' : {'max_depth':[5, 6, 7]},
             'case_2' : {'min_samples_split': [10, 15, 20]},
             'case_3' : {'max_features': ['sqrt', 'log2']}
            }

In [10]:
x_cols = ['A2', 'A3', 'A8', 'A11', 'A14', 'A15', 'A1_a', 'A1_b', 'A4_u', 'A4_y',
          'A5_gg', 'A5_p', 'A6_aa', 'A6_c', 'A6_cc', 'A6_d', 'A6_e', 'A6_ff',
          'A6_i', 'A6_j', 'A6_k', 'A6_m', 'A6_q', 'A6_r', 'A6_w', 'A6_x', 'A7_bb',
          'A7_dd', 'A7_ff', 'A7_h', 'A7_j', 'A7_n', 'A7_o', 'A7_v', 'A7_z',
          'A9_t', 'A10_t', 'A12_t', 'A13_p', 'A13_s'
         ]
X = df_train[x_cols]
y = df_train['target']

In [11]:
# create a data frame, to store results
df_accuracy = pd.DataFrame(data={'case': [], 
                                 'tunining_parameter': [], 
                                 'value':[],
                                 'training_accuracy':[],
                                 'test_accuracy':[]
                                }
                          )

for case in dict_comb.keys():
    for tuning_parameter in dict_comb[case].keys():
        for value_parameter in dict_comb[case][tuning_parameter]:
            if tuning_parameter == "max_depth":
                model_decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', max_depth = value_parameter)
            elif tuning_parameter == "min_samples_split":
                model_decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', 
                                                             min_samples_split = value_parameter)
            elif tuning_parameter == "max_features":
                model_decision_tree = DecisionTreeClassifier(random_state=0, criterion='gini', max_features = value_parameter)
            else:
                print(f'We have not this parameter for testing {tuning_parameter}')
            model_decision_tree.fit(X, y)
            train_accuracy = np.round(model_decision_tree.score(df_train[x_cols], df_train['target']) * 100, 2)
            test_accuracy = np.round(model_decision_tree.score(df_test[x_cols], df_test['target']) * 100, 2)
            df_accuracy.loc[len(df_accuracy.index)] = [case, tuning_parameter, value_parameter, train_accuracy, test_accuracy]
df_accuracy

,case,tunining_parameter,value,training_accuracy,test_accuracy
0,case_1,max_depth,5.0,90.98,89.22
1,case_1,max_depth,6.0,92.79,88.02
2,case_1,max_depth,7.0,93.79,87.43
3,case_2,min_samples_split,10.0,95.79,86.23
4,case_2,min_samples_split,15.0,93.99,85.63
5,case_2,min_samples_split,20.0,92.79,86.23
6,case_3,max_features,sqrt,100.00,76.05
7,case_3,max_features,log2,100.00,80.24
